In [12]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 [==============================] - 113s 1us/step


In [3]:
x_train.shape # (50000, 32, 32, 3)
y_train.shape # (50000, 1)

(50000, 1)

In [6]:
x_train = x_train.astype('float') / 255
x_test = x_test.astype('float') / 255

In [8]:
id2class = {
    idx: name for idx, name in enumerate([
        'airplane', 'automobile', 'bird',
        'cat', 'deer', 'dog', 'frog',
        'horse', 'ship', 'truck'
    ])
}
print(id2class)

{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [19]:
def display_cifar_images(
    x: np.ndarray,
    y: np.ndarray,
    id2class: dict[int, str],
    n: int = 4
):
    random_images_indices = np.random.randint(0, x.shape[0], n)
    images = x[random_images_indices]
    classes = [id2class[y[idx, 0]] for idx in random_images_indices]


    f, axs = plt.subplots(1, n, figsize=((3*n, 3)))
    for image, name, ax in zip(images, classes, axs):
        ax.imshow(image)
        ax.axis('off')
        ax.set(title=f"Class: {name}")
    plt.show()

In [1]:
display_cifar_images(x_train, y_train, id2class)

NameError: name 'display_cifar_images' is not defined

In [24]:
class NNClassifier(tf.keras.Model):
    """Image classifier with fully-connected layers"""
    def __init__(self, hidden_dim: int, n_classes: int, *args, **kwargs):
        super(NNClassifier, self).__init__(*args, **kwargs) # ініціалізація супер класу є обов'язкова!


        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=hidden_dim, activation='relu')
        self.dense2 = tf.keras.layers.Dense(units=hidden_dim, activation='relu')
        self.out = tf.keras.layers.Dense(n_classes) # можна додати activation = 'softmax'

    def call(self, inputs, training: bool):
        x = self.flatten(inputs)
        x = self.dense2(self.dense1(x))
        return self.out(x)

model = NNClassifier(hidden_dim=100, n_classes=len(id2class))


In [25]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
    # спарс версія оскільки y_train та y_test містять номери класів
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [28]:
hist = model.fit(
    x=x_train,
    y=y_train,
    epochs=3,
    batch_size=128,
    validation_data=(x_test, y_test)
)


Epoch 1/3
391/391 [==============================] - 2s 6ms/step - loss: 1.8812 - accuracy: 0.3338 - val_loss: 1.8538 - val_accuracy: 0.3449
Epoch 2/3
391/391 [==============================] - 2s 5ms/step - loss: 1.8351 - accuracy: 0.3545 - val_loss: 1.8229 - val_accuracy: 0.3629
Epoch 3/3
391/391 [==============================] - 2s 5ms/step - loss: 1.8113 - accuracy: 0.3680 - val_loss: 1.7971 - val_accuracy: 0.3707


In [29]:
model.summary()

Model: "nn_classifier_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         multiple                  0         
                                                                 
 dense_6 (Dense)             multiple                  307300    
                                                                 
 dense_7 (Dense)             multiple                  10100     
                                                                 
 dense_8 (Dense)             multiple                  1010      
                                                                 
Total params: 318410 (1.21 MB)
Trainable params: 318410 (1.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
hist.history

{'loss': [1.8812212944030762, 1.8351383209228516, 1.8113430738449097],
 'accuracy': [0.3338199853897095, 0.3545199930667877, 0.3679800033569336],
 'val_loss': [1.8537545204162598, 1.822855830192566, 1.7971023321151733],
 'val_accuracy': [0.3449000120162964, 0.3628999888896942, 0.37070000171661377]}

In [32]:
test_loss, test_acc = model.evaluate(x_test, y_test)
test_loss, test_acc

313/313 [==============================] - 1s 3ms/step - loss: 1.7971 - accuracy: 0.3707


(1.7971025705337524, 0.37070000171661377)